!pip install futu-api

!pip install yfinance

https://openapi.moomoo.com/futu-api-doc/en/trade/base.html

https://www.moomoo.com/download/OpenAPI?_ga=2.209413910.5081162.1671690607-91384723.1670440254

In [1]:
import yfinance
from futu import *

In [2]:
daily_amount = 1000

# build account

https://openapi.moomoo.com/futu-api-doc/en/quick/opend-base.html

https://openapi.moomoo.com/futu-api-doc/en/trade/base.html

https://openapi.moomoo.com/futu-api-doc/en/trade/trade.html#9434

In [3]:
trd_ctx = OpenSecTradeContext(
    filter_trdmarket=TrdMarket.US, 
    host='127.0.0.1', port=11111, 
    security_firm = SecurityFirm.FUTUINC,
    )

2022-12-22 12:11:34,704 | 20724 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=70981587


# calculate today's investment

## query historical trading data

https://openapi.moomoo.com/futu-api-doc/en/trade/get-history-order-list.html

In [4]:
%%time

ret, data = trd_ctx.history_order_list_query(
    code = 'US.QQQ'
    )

CPU times: total: 0 ns
Wall time: 1.45 s


In [5]:
data['amount'] = data['qty'] * data['dealt_avg_price']

## expected amount of investment

In [6]:
data['create_timestamp']= pd.to_datetime(data['create_time'])
data['create_date'] = data['create_timestamp'].apply(lambda x: x.strftime('%Y-%m-%d'))
invested_days = data['create_date'].unique().size

In [7]:
expected_total_investment_amount = (invested_days+1)*daily_amount

invested_amount = data['amount'].sum()

expected_amount_today = expected_total_investment_amount - invested_amount

## query current price 

In [8]:
market_price = yfinance.Ticker('QQQ').info['regularMarketPrice']

## shares to by

In [9]:
shares_to_buy = expected_amount_today/market_price

shares_to_buy = round(shares_to_buy)

In [10]:
print(f'shares_to_buy: {shares_to_buy}')

shares_to_buy: 5


# trading

https://openapi.moomoo.com/futu-api-doc/en/trade/place-order.html

pwd_unlock = '123456'

ret, data = trd_ctx.unlock_trade(pwd_unlock)  

ret, data = trd_ctx.place_order(
    price=510.0, 
    qty=shares_to_buy,
    code='US.QQQ', 
    trd_side=TrdSide.BUY, 
    trd_env=TrdEnv.SIMULATE)

# close the session

In [11]:
trd_ctx.close()

2022-12-22 12:11:38,050 | 20724 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


# end